<a href="https://colab.research.google.com/github/bhartiansh/cnn_pruning_cifar10/blob/main/Effective_Automatic_Channel_Pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/bhartiansh/cnn_pruning_cifar10.git
%cd cnn_pruning_cifar10

fatal: destination path 'cnn_pruning_cifar10' already exists and is not an empty directory.
/content/cnn_pruning_cifar10


In [2]:
!pip install -q tensorflow-model-optimization

In [3]:
import tensorflow as tf
import numpy as np
from sklearn.cluster import KMeans
from models.resnet20 import build_resnet20  # Your custom ResNet-20 model

In [4]:
# -------------------------------
# Step 1: Cluster filters using cosine similarity
# -------------------------------
def cluster_filters(layer_weights, num_clusters):
    filters = layer_weights.reshape(layer_weights.shape[-1], -1)
    filters_norm = np.linalg.norm(filters, axis=1, keepdims=True)
    filters_normalized = filters / (filters_norm + 1e-8)
    kmeans = KMeans(n_clusters=num_clusters, random_state=0, n_init='auto')
    return kmeans.labels_

In [10]:
from sklearn.cluster import KMeans
import numpy as np

def cluster_filters(layer_weights, num_clusters):
    # Flatten each filter (kernel) to 1D
    filters = layer_weights.reshape(-1, layer_weights.shape[-1])
    filters = filters.transpose()  # Shape: (num_filters, filter_size)

    # Normalize filters
    filters_norm = np.linalg.norm(filters, axis=1, keepdims=True)
    filters_normalized = filters / (filters_norm + 1e-8)

    # Fit KMeans and return labels
    kmeans = KMeans(n_clusters=num_clusters, random_state=0, n_init='auto')
    kmeans.fit(filters_normalized)
    return kmeans.labels_


In [11]:
def train_eacp_model(x_train, y_train, x_val, y_val,
                     sparsity=0.5, epochs=50, batch_size=128):
    # Build and compile model
    model = build_resnet20()
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Apply EACP pruning
    print(f"✂️  Applying EACP pruning with sparsity = {sparsity:.2f}...")
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Conv2D):
            weights = layer.get_weights()[0]
            num_filters = weights.shape[-1]
            num_clusters = max(1, int(num_filters * (1 - sparsity)))

            if num_clusters < num_filters:
                cluster_labels = cluster_filters(weights, num_clusters)
                prune_filters(layer, cluster_labels)

    # Fine-tune pruned model
    print("🔁 Fine-tuning pruned model...")
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))

    return model


In [ ]:
# Load CIFAR-10 (unchanged)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Train with EACP (ResNet-20)
model = train_eacp_model(x_train, y_train, x_test, y_test, sparsity=0.3, epochs=30, batch_size=64)

✂️  Applying EACP pruning with sparsity = 0.30...
🔁 Fine-tuning pruned model...
Epoch 1/30
782/782 [==============================] - 647s 815ms/step - loss: 1.3517 - accuracy: 0.5059 - val_loss: 1.7901 - val_accuracy: 0.4385
Epoch 2/30
782/782 [==============================] - 625s 799ms/step - loss: 0.9255 - accuracy: 0.6716 - val_loss: 1.1085 - val_accuracy: 0.6321
Epoch 3/30
782/782 [==============================] - 618s 791ms/step - loss: 0.7536 - accuracy: 0.7379 - val_loss: 0.8986 - val_accuracy: 0.6988
Epoch 4/30
532/782 [===================>..........] - ETA: 3:05 - loss: 0.6558 - accuracy: 0.7732

In [ ]:
# Load CIFAR-10 (unchanged)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Train with EACP (ResNet-20)
model = train_eacp_model(x_train, y_train, x_test, y_test, sparsity=0.5, epochs=30, batch_size=64)

In [ ]:
# Load CIFAR-10 (unchanged)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Train with EACP (ResNet-20)
model = train_eacp_model(x_train, y_train, x_test, y_test, sparsity=0.7, epochs=30, batch_size=64)

In [ ]:
# 1. Force compatible versions (reset everything to Colab defaults)
!pip install -U --force-reinstall numpy==1.23.5
!pip install -U --force-reinstall tensorflow==2.14.0
!pip install -U tensorflow-model-optimization

# 2. Restart runtime automatically after install
import os
os.kill(os.getpid(), 9)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 26.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
xarray 2025.1.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
bigframes 1.42.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 wh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 258.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 73.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
  Attempting uninstall: absl-py
    Found existing installation: absl-py 2.2.2
    Uninstalling absl-py-2.2.2:
      Successfully uninstalled absl-py-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.14.0 which is incompatible